In [1]:
import psycopg2

In [2]:
conn = psycopg2.connect("host=127.0.0.1 port=5433 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

## Query to look at the top five artists with the most song plays for the entire history of the data (currently returns nothing because none of the songs played are referenced in the artists or songs tables):

In [4]:
cur.execute(
    """
    SELECT
        songplays.artist_id,
        artists.name,
        COUNT(DISTINCT(songplays.songplay_id))
    FROM
        songplays
    INNER JOIN artists USING (artist_id)
    GROUP BY
        songplays.artist_id,
        artists.name
    """
)
results = cur.fetchall()
results

[]

## Query to look at songplays by hour of the day:

In [5]:
cur.execute(
    """
    SELECT
        time.hour,
        COUNT(DISTINCT(songplays.songplay_id))
    FROM
        songplays
    INNER JOIN time USING (start_time)
    GROUP BY time.hour
    """
)
results = cur.fetchall()
results

[(0.0, 155),
 (1.0, 154),
 (2.0, 117),
 (3.0, 109),
 (4.0, 136),
 (5.0, 162),
 (6.0, 183),
 (7.0, 179),
 (8.0, 207),
 (9.0, 270),
 (10.0, 312),
 (11.0, 336),
 (12.0, 308),
 (13.0, 324),
 (14.0, 432),
 (15.0, 477),
 (16.0, 542),
 (17.0, 494),
 (18.0, 498),
 (19.0, 367),
 (20.0, 360),
 (21.0, 280),
 (22.0, 217),
 (23.0, 201)]

In [11]:
hourly_data = list(zip(*results))
hourly_data

[(0.0,
  1.0,
  2.0,
  3.0,
  4.0,
  5.0,
  6.0,
  7.0,
  8.0,
  9.0,
  10.0,
  11.0,
  12.0,
  13.0,
  14.0,
  15.0,
  16.0,
  17.0,
  18.0,
  19.0,
  20.0,
  21.0,
  22.0,
  23.0),
 (155,
  154,
  117,
  109,
  136,
  162,
  183,
  179,
  207,
  270,
  312,
  336,
  308,
  324,
  432,
  477,
  542,
  494,
  498,
  367,
  360,
  280,
  217,
  201)]

In [25]:
# Turn it into a graph using plotly
import plotly.express as px
fig = px.bar(x=hourly_data[0],y=hourly_data[1],
             title="Hourly Songplays (All Time)")
fig.update_xaxes(title="Hour")
fig.update_xaxes(tickvals=list(range(0,23)))
fig.update_yaxes(title="Songplays")
fig.show()